## Initialization

In [5]:
import sys
#sys.path.append('/Users/chandrasekharmukherjee/Home/Centrality framework/Modules')
sys.path.append('I:/내 드라이브/backup/document/USC/Research/MCPC/Code/Codes')

#Import packages
#imports
import time
import pandas as pd
from collections import Counter
import sknetwork


from sknetwork.ranking import PageRank
from sknetwork.ranking import Betweenness
from sknetwork.ranking import Closeness

#import umap as umap

from numba.typed import List
import warnings
from numba import njit
import pynndescent
import numpy as np
from sklearn.cluster import SpectralClustering
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import matplotlib
import operator
from sklearn.utils.extmath import randomized_svd
from random import randint
from sklearn.utils.extmath import randomized_svd
from sklearn.decomposition import TruncatedSVD
import scipy
#from umap.umap_ import *
import math
from random import randint
import keras
from keras.datasets import mnist
from keras.datasets import cifar10
from keras.datasets import cifar100
from keras.datasets import fashion_mnist
import scanpy
from sklearn.metrics.cluster import normalized_mutual_info_score
from sklearn.metrics import adjusted_mutual_info_score, roc_auc_score
from sklearn.metrics.cluster import adjusted_rand_score
from sklearn.metrics.cluster import adjusted_mutual_info_score

#import community as community_louvain
from sklearn.cluster import KMeans
from sklearn.metrics import fowlkes_mallows_score

import scipy

import igraph 
import networkx as nx


import metric as met 
import embedding as embed   
import data_utils_ch as data_util

%load_ext autoreload
%autoreload 2
import importlib

import metric as met 
import embedding as embed   
import data_utils_ch as data_util
from subgraph import *


met=importlib.reload(met)
embed=importlib.reload(embed)
data_util=importlib.reload(data_util)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Testing every dataset with top k % nodes induced subgraph (Louvain)

### Top k% induced subgraph + Strong Majority Vote later

##### PDF Export version (Final Working Version)

In [29]:
import Cust_Top as Cust
import networkx.algorithms.community.louvain as Louv
from sklearn.metrics.cluster import normalized_mutual_info_score as NMI
from collections import deque
from matplotlib.backends.backend_pdf import PdfPages
import os
from subgraph import * 
import time 
import debug
from collections import defaultdict

res_=[] #0.26, 0.24 is best for Louvain, 3.3, 2.0
labels=[]
total_labels=[]
names = []

data_names = ['Zhengmix8eq','mRNA','miRNA','FashionMNIST','MNIST','seeds','breast-cancer','Omniglot','bbc_news','20NewsGroups_tfdif','biorxiv','big_patent','Cora','Cora full','Citeseer','Eu core']
#data_names = ['mRNA','miRNA','FashionMNIST','MNIST','seeds','breast-cancer','Omniglot','bbc_news','20NewsGroups_tfdif','biorxiv','big_patent','Cora','Cora full','Citeseer','Eu core']
#data_names = ['mRNA']
#data_names = ['Zhengmix8eq']
'''
datasets with nodes that have no outgoing edges : 
['Cora full', 'Citeseer', 'Eu core']
'''
#data_names = ['Cora full', 'Citeseer', 'Eu core']
# FR_ord = ['Rd', 'Ord']
# FR_Recalc = ['FR_avg', 'FR_Re']
FR_tp = ['FL','FL_ng','FL_ng_prop','PageRank']
FR_tp_short = ['FL','FL_ng','FL_ng_p','PR']
Num_hops = {0:'5',1:'log(n)'}
Mod_tp = {0: 'Louv'}

#top k% nodes for induced subgraph
k=0.2
traverse = {0:'Reset',1:'Static'}
ord = {0:'Rand',1:'FR_order'}

#list of plotted figures
figures = []

for data in data_names: 
    pdf_name = './Plots/'+data+'/fixed_'+data+'.pdf'
    
    
    for survive in [0]:
        '''
        Get the Graph from Edge_List and get the labels
        '''
        #DO not recalculate Graph for non bulk RNA datasets
        if data not in ['mRNA','miRNA'] and survive==1:
            continue
        G, label= data_to_graph(data,survive)
        cluster_sizes = defaultdict(int)
        for node in G.nodes():
            cluster_sizes[label[node]]+=1
        #print('Selected Labels:',selected_labels_dict)
        #add self loop to nodes without outgoing edges
        #G_ is copy of G
        G_ = G.copy()
        # if (data in ['Cora full', 'Citeseer', 'Eu core']):
        #     for node in G.nodes():
        #         if not list(G.successors(node)):
        #             G_.remove_node(node)
        for node in G_.nodes():
            if not list(G_.successors(node)):
                #print(data, ' has a node without outgoing edges:',node)
                G_.add_edge(node, node, weight=1)
        print('Number of nodes:',G.number_of_nodes())
        

        data_directory = f'./Plots/{data}'
        file_path = f'{data_directory}/{data}.txt'
        os.makedirs(data_directory, exist_ok=True)
        with open(file_path, 'w') as f:
            f.write('------------------------------------------------------------------------\n')
        
        
            
        for Num_hop in [1]:
            plt.figure(figsize=(10,15)) # figure for NMI vs purity
            plt.figure(figsize=(10,6)) # figure for NMI vs Purity (whole)
            plt.figure(figsize=(10,15)) # figure for NMI
            plt.figure(figsize=(10,15)) # figure for Purity
            plt.figure(figsize=(10,15)) # figure for preserve_ratio
            plt.figure(figsize=(10,15)) # figure for balancedness
            plt.figure(figsize=(10,15)) # figure for InEdge Degrees
            #List of FR_values
            FR_precalculated = []
            for fr_type in range(4): 
                '''
                Calculate FlowRank for the whole graph
                '''
                if Num_hop==0:
                    node2FR = calc_FlowRank(G_, fr_type, 5)
                else:
                    node2FR = calc_FlowRank(G_, fr_type, np.log2(G.number_of_nodes()))
                FR_precalculated.append(node2FR)

            for idx, res in enumerate([0.05,0.25, 0.5, 1, 1.5, 5]):
            #for idx, res in enumerate([0.05]):
            #for idx, res in enumerate([0.01,0.03,0.05,0.1,0.15,0.2]):
                for Mod_type in [0]:
                    ''' 
                    Louvain on G (baseline)
                    '''
                    #start_time = time.time()
                    partition = debug.louvain_partitions(G, seed=0,resolution=res)
                    partition_ = deque(partition, maxlen=1).pop()
                    label_ = part_to_full_label(partition_,G.number_of_nodes())
                    NMI_baseline = NMI(label_, label)
                    Purity_baseline = met.purity_score(label, label_)
                    
                    #print('Time Louvain on whole G:', time.time()-start_time)
                    for FR_type in range(4):
                        name =  data + ' (top ' + str(k*100) + '%): '+ str(FR_tp[FR_type]) + ' + '+ str(Num_hops[Num_hop]) +' hops'
                        if FR_type==3:
                            name = data + ' (top ' + str(k*100) + '%): PageRank'
                        if data in ['mRNA','miRNA']:
                            name = data +  ' (top ' + str(k*100) + '%): '+ str(FR_tp[FR_type]) + ' + '+ str(Num_hops[Num_hop]) +' hops' + ' + Survive: '+str(survive)
                            if FR_type==3:
                                name = data + ' (top ' + str(k*100) + '%): PageRank' + ' + Survive: '+str(survive)
                        print(' ---',name,'| res: ',str(res))
                        
                        # Create the directory if it doesn't exist
                        os.makedirs(data_directory, exist_ok=True)
                        with open(file_path, 'a') as f:
                            f.write(' ---'+name+'| res: '+str(res)+'\n')
                            f.write('NMI baseline: ' + str(round(NMI_baseline,3)) + '\n')
                        

                        '''
                        Get induced subgraph of top k% nodes in FR value
                        '''
                        node2FR = FR_precalculated[FR_type]
                        
                        # if data in ['Cora full', 'Citeseer', 'Eu core']:
                        #     for node in G.nodes():
                        #         if not list(G.successors(node)):
                        #             node2FR[node]=0

                        node_list = sorted(G.nodes, key=lambda x: node2FR[x], reverse=True)
                        #print('Time sort:', time.time()-start_time)
                        
                        k_ = int(k*len(node_list))
                        top_nodes = node_list[:k_]
                        H = getInducedSubgraph(G, top_nodes) # H = induced subgraph
                        #print('Time Induced Subgraph:', time.time()-start_time)
                        
                        '''
                        label counts for balancedness calculation
                        '''
                        selected_labels_dict = defaultdict(int)
                        for node in H.nodes():
                            selected_labels_dict[label[node]]+=1
                        

                        '''
                        Louvain on the subgraph H 
                        '''
                        # partition = RC10.louvain_partitions(H, seed=0,resolution=res, FR_type=FR_type, Mod_type=Mod_type)
                        #partition = Louv.louvain_partitions(H, seed=0,resolution=res ,threshold=0.0001)
                        
                        partition = debug.louvain_partitions(H, seed=0,resolution=res)
                        partition_ = deque(partition, maxlen=1).pop()
                        H_label = part_to_full_label(partition_,G.number_of_nodes())
                        #print('Time Louvain:', time.time()-start_time)
        
                        '''Run Louvain Again on the rest ( Warm Start )'''
                        #start_time = time.time()

                        k_ = int(k*len(node_list))
                        nodes_rest = node_list[k_:]
                        
                        for node in nodes_rest:
                            partition_.append({node})
                        part = Cust.louvain_partitions(G, seed=0,resolution=res,init_part=partition_)
                        part_ = deque(part, maxlen=1).pop()
                        label_ = part_to_full_label(part_,G.number_of_nodes())
                        NMI_twice = NMI(label_, label)
                        Purity_twice = met.purity_score(label, label_)
                        #print('Time Louvain twice:', time.time()-start_time)
                        print('NMI baseline:', round(NMI_baseline,3))
                        print('NMI Louvain twice:', round(NMI_twice,3))


                        '''
                        Add nodes to the subgraph by strong majority vote
                        '''
                        #See the trend by adding node to the subgraph by strong majority vote (in order of FlowRank Value)
                        
                        #print('H_label: ',H_label, 'len: ',len(H_label))
                        
                        
                        #start_time = time.time()
                        NMI_List, Purity_List, Balance_List, Preserv_List, InEdge_List = merge_by_vote(top_nodes, nodes_rest, H_label, G, label, selected_labels_dict, cluster_sizes)
                        #print('Time merge by vote:', time.time()-start_time)
                        
                        #Plot NMI_List
                        fig_numbers = plt.get_fignums()
                        NMI_vs_Purity = fig_numbers[-7]
                        NMI_vs_Purity_whole = fig_numbers[-6]
                        nmi_fig =fig_numbers[-5]
                        purity_fig = fig_numbers[-4]
                        preserve_fig = fig_numbers[-3]
                        balance_fig = fig_numbers[-2]
                        InEdge_fig = fig_numbers[-1]
                        

                        max_dot_size = 100
                        #plot NMI vs Purity
                        markers = ['o','s','*','^','h','D'] #idx (resolution)
                        colors = ['b','g','y','r'] #FR_type
                        plt.figure(NMI_vs_Purity)
                        plt.subplot(3,2,idx+1)
                        percent_nodes = (H.number_of_nodes() + len(NMI_List))*100 / G.number_of_nodes()
                        #plt.scatter(Purity_List[-1], NMI_List[-1], color = colors[FR_type], s = percent_nodes*max_dot_size, marker = markers[idx],label = str(FR_tp[FR_type])+ '|'+str(round(percent_nodes,2))+ '% nodes')
                        plt.scatter(Purity_List[-1], NMI_List[-1], color = colors[FR_type], s = percent_nodes/100*max_dot_size,label = str(FR_tp[FR_type])+ '|'+str(round(percent_nodes,2))+ '% nodes')
                        if FR_type==0:
                            plt.scatter(Purity_baseline, NMI_baseline, color = 'k', s = max_dot_size,label = 'Louv_baseline')
                            #plt.scatter(Purity_baseline, NMI_baseline, color = 'k', s = max_dot_size*0.5, marker = markers[idx],label = 'Louv_baseline|'+str(res))
                        #plt.scatter(Purity_twice, NMI_twice, color = 'm', s = max_dot_size, marker = markers[idx],label = '2*Louv|'+str(res))

                        #plot NMI vs Purity (whole)
                        plt.figure(NMI_vs_Purity_whole)
                        plt.scatter(Purity_List[-1], NMI_List[-1], color = colors[FR_type], s = percent_nodes/100*max_dot_size, marker = markers[idx],label = str(FR_tp[FR_type])+ '|'+str(res)+'|'+str(round(percent_nodes,2))+ '% nodes')
                        if FR_type==0:
                            plt.scatter(Purity_baseline, NMI_baseline, color = 'k', s = max_dot_size, marker = markers[idx],label = 'Louv_baseline|'+str(res))
                        #plt.scatter(Purity_twice, NMI_twice, color = 'm', s = max_dot_size, marker = markers[idx],label = '2*Louv|'+str(res))

                        #plot NMI
                        plt.figure(nmi_fig)
                        plt.subplot(3, 2, idx+1)
                        plt.plot(NMI_List,  color = colors[FR_type], label = str(FR_tp[FR_type]))

                        if FR_type==0: 
                            #Horizontal Line for Baselines
                            NMI_baseline_list = [NMI_baseline]*len(nodes_rest)
                            plt.plot(NMI_baseline_list, label = 'Louv_baseline')
                        NMI_Louv_twice_list = [NMI_twice]*len(nodes_rest)
                        plt.plot(NMI_Louv_twice_list, label = '2*Louv|'+str(FR_tp_short[FR_type]))
                        
                        with open(file_path, 'a') as f:
                            f.write('# of nodes not included: ' + str(G.number_of_nodes() - H.number_of_nodes() - len(NMI_List)+1) + '\n')
                            f.write('NMI_Louvain_twice: ' + str(round(NMI_twice,3)) + '\n')
                            f.write('NMI after top ' + str(k*100) + '% nodes: ' + str(round(NMI_List[0],3)) + '\n')
                            f.write('Max NMI: ' + str(round(max(NMI_List),3)) + '\n')
                        print('Number of nodes not included: ' , G.number_of_nodes() - H.number_of_nodes() - len(NMI_List)+1) 
                        print('NMI after top ', k*100, '% nodes:', round(NMI_List[0],3))
                        print('Max NMI: ', round(max(NMI_List),3))

                        #plot Purity
                        plt.figure(purity_fig)
                        plt.subplot(3, 2, idx+1)
                        plt.plot(Purity_List,  color = colors[FR_type], label = str(FR_tp[FR_type]))
                        if FR_type==0: 
                            #Horizontal Line for Baselines
                            Purity_baseline_list = [Purity_baseline]*len(nodes_rest)
                            plt.plot(Purity_baseline_list, label = 'Louv_baseline')
                    
                        Purity_Louv_twice_list = [Purity_twice]*len(nodes_rest)
                        plt.plot(Purity_Louv_twice_list, label = '2*Louv|'+str(FR_tp_short[FR_type]))
                        
                        with open(file_path, 'a') as f:
                            f.write('Purity_Louvain_twice: ' + str(round(Purity_twice,3)) + '\n')
                            f.write('Purity after top ' + str(k*100) + '% nodes: ' + str(round(Purity_List[0],3)) + '\n')
                            f.write('Max Purity: ' + str(round(max(Purity_List),3)) + '\n')

                        #plot preserve_ratio
                        plt.figure(preserve_fig)
                        plt.subplot(3, 2, idx+1)
                        plt.plot(Preserv_List,  color = colors[FR_type], label = str(FR_tp[FR_type]))

                        #plot balancedness
                        plt.figure(balance_fig)
                        plt.subplot(3, 2, idx+1)
                        plt.plot(Balance_List,  color = colors[FR_type], label = str(FR_tp[FR_type])) 

                        #plot InEdge Degrees
                        plt.figure(InEdge_fig)
                        plt.subplot(3, 2, idx+1)
                        plt.plot(InEdge_List,  color = colors[FR_type], label = str(FR_tp[FR_type]))
                    
                plt.figure(NMI_vs_Purity)
                plt.tight_layout()
                plt.subplot(3,2,idx+1)
                plt.xlabel('Purity')
                plt.ylabel('NMI')
                plt.title(data + ' | top ' + str(k*100) + '%' + '| Num_hops: ' + str(Num_hops[Num_hop]) + '|res: ' + str(res)+ '|')
                #plt.legend()
                plt.legend()

                plt.figure(NMI_vs_Purity_whole)
                plt.xlabel('Purity')
                plt.ylabel('NMI')
                plt.title(data + ' | top ' + str(k*100) + '%' + '| Num_hops: ' + str(Num_hops[Num_hop]))
                #plt.legend()
                plt.legend(loc='center left', bbox_to_anchor=(1, 0.5), fontsize=8)
                plt.tight_layout(rect=[0, 0, 0.85, 1])

                plt.figure(nmi_fig)
                plt.tight_layout()
                plt.subplot(3, 2, idx+1)    
                #Plot all at once
                plt.xlabel('Number of nodes added after top ' + str(k*100) + '% nodes')
                plt.ylabel('NMI')
                plt.title(data + ' | top ' + str(k*100) + '%' + '| Num_hops: ' + str(Num_hops[Num_hop]) + '|res: ' + str(res)+ '|')
                plt.legend()

                plt.figure(purity_fig)
                plt.tight_layout()
                plt.subplot(3, 2, idx+1)
                #Plot all at once
                plt.xlabel('Number of nodes added after top ' + str(k*100) + '% nodes')
                plt.ylabel('Purity')
                plt.title(data + ' | top ' + str(k*100) + '%' + '| Num_hops: ' + str(Num_hops[Num_hop]) + '|res: ' + str(res)+ '|')
                plt.legend()

                plt.figure(preserve_fig)
                plt.tight_layout()
                plt.subplot(3, 2, idx+1)
                #Plot all at once
                plt.xlabel('Number of nodes added after top ' + str(k*100) + '% nodes')
                plt.ylabel('Preserve Ratio')
                plt.title(data + ' | top ' + str(k*100) + '%' + '| Num_hops: ' + str(Num_hops[Num_hop]) + '|res: ' + str(res)+ '|')
                plt.legend()

                plt.figure(balance_fig)
                plt.tight_layout()
                plt.subplot(3, 2, idx+1)
                #Plot all at once
                plt.xlabel('Number of nodes added after top ' + str(k*100) + '% nodes')
                plt.ylabel('Balancedness')
                plt.title(data + ' | top ' + str(k*100) + '%' + '| Num_hops: ' + str(Num_hops[Num_hop]) + '|res: ' + str(res)+ '|')
                plt.legend()

                plt.figure(InEdge_fig)
                plt.tight_layout()
                plt.subplot(3, 2, idx+1)
                #Plot all at once
                plt.xlabel('Number of nodes added after top ' + str(k*100) + '% nodes')
                plt.ylabel('InEdge Degrees')
                plt.title(data + ' | top ' + str(k*100) + '%' + '| Num_hops: ' + str(Num_hops[Num_hop]) + '|res: ' + str(res)+ '|')
                plt.legend()
    
    
    p = PdfPages(pdf_name)            
    fig_nums = plt.get_fignums() 
    figs = [plt.figure(n) for n in fig_nums] 
	
	# iterating over the numbers in list 
    for fig in figs: 
		# and saving the files 
        fig.savefig(p, format='pdf') 
	
	# close the object 
    p.close() 
    plt.close('all')
    
        
            

                                
                    

                    
                    



Zhengmix8eq 3994
Log transform done
(3994, 50)
Accuracy of  15 -NN graph is 0.793
59910
Number of nodes: 3994
 --- Zhengmix8eq (top 20.0%): FL + log(n) hops | res:  0.05
NMI baseline: 0.688
NMI Louvain twice: 0.691
Number of nodes not included:  89
NMI after top  20.0 % nodes: 0.83
Max NMI:  0.833
 --- Zhengmix8eq (top 20.0%): FL_ng + log(n) hops | res:  0.05
NMI baseline: 0.688
NMI Louvain twice: 0.688
Number of nodes not included:  16
NMI after top  20.0 % nodes: 0.841
Max NMI:  0.849
 --- Zhengmix8eq (top 20.0%): FL_ng_prop + log(n) hops | res:  0.05
NMI baseline: 0.688
NMI Louvain twice: 0.687
Number of nodes not included:  15
NMI after top  20.0 % nodes: 0.731
Max NMI:  0.734
 --- Zhengmix8eq (top 20.0%): PageRank | res:  0.05
NMI baseline: 0.688
NMI Louvain twice: 0.69
Number of nodes not included:  102
NMI after top  20.0 % nodes: 0.711
Max NMI:  0.717
 --- Zhengmix8eq (top 20.0%): FL + log(n) hops | res:  0.25
NMI baseline: 0.782
NMI Louvain twice: 0.801
Number of nodes not inc

KeyboardInterrupt: 